In [57]:
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import pickle

In [72]:
class StutterDetector:
    def __init__(self):
        self.scaler = StandardScaler()
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)

    def extract_features(self, audio_path):
        """Extract features from the audio file."""
        try:
            y, sr = librosa.load(audio_path, duration=5)  # Load audio (5 seconds max)
            
            # Extract features
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
            zero_crossing_rate = librosa.feature.zero_crossing_rate(y)

            # Calculate statistics for each feature
            features = {
                'mfcc_mean': np.mean(mfccs, axis=1),
                'mfcc_std': np.std(mfccs, axis=1),
                'spectral_centroid_mean': np.mean(spectral_centroid),
                'spectral_centroid_std': np.std(spectral_centroid),
                'zero_crossing_rate_mean': np.mean(zero_crossing_rate),
                'zero_crossing_rate_std': np.std(zero_crossing_rate)
            }

            # Flatten to 1D array
            feature_vector = []
            for value in features.values():
                if isinstance(value, np.ndarray):
                    feature_vector.extend(value)
                else:
                    feature_vector.append(value)

            return np.array(feature_vector)
        except Exception as e:
            print(f"Error extracting features from {audio_path}: {str(e)}")
            return None

    def train(self, audio_files, labels):
        """Train the model on the provided audio files and labels."""
        features = []
        for audio_file in audio_files:
            feature_vector = self.extract_features(audio_file)
            if feature_vector is not None:
                features.append(feature_vector)

        # Convert to numpy array
        X = np.array(features)
        y = np.array(labels)

        # Scale features
        X_scaled = self.scaler.fit_transform(X)

        # Split data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

        # Train model
        self.model.fit(X_train, y_train)

        # Print accuracy
        train_accuracy = self.model.score(X_train, y_train)
        test_accuracy = self.model.score(X_test, y_test)
        print(f"Training Accuracy: {train_accuracy:.2f}")
        print(f"Testing Accuracy: {test_accuracy:.2f}")

    def predict(self, audio_path):
        """Predict whether the given audio contains stuttering and provide recommendations."""
        try:
            # Extract features
            features = self.extract_features(audio_path)

            if features is None:
                return {"error": "Error extracting features from the audio file."}

            # Scale features
            features_scaled = self.scaler.transform(features.reshape(1, -1))

            # Make prediction
            prediction = self.model.predict(features_scaled)
            probability = self.model.predict_proba(features_scaled)

            # Generate recommendations based on prediction
            if prediction[0] == 1:  # Stuttering detected
                result = {
                    "prediction": "stutter",
                    "confidence": max(probability[0]),
                    "recommendations": [
                        "Practice breathing exercises to control airflow during speech.",
                        "Consider consulting a speech-language pathologist for therapy.",
                        "Speak slowly and deliberately to improve fluency.",
                        "Try practicing with apps designed for speech improvement."
                    ]
                }
            else:  # Fluent speech
                result = {
                    "prediction": "fluent",
                    "confidence": max(probability[0]),
                    "recommendations": [
                        "Maintain confidence in your speech.",
                        "Continue practicing speaking regularly to maintain fluency.",
                        "Explore public speaking to further improve fluency and confidence."
                    ]
                }

            return result
        except Exception as e:
            print(f"Error during prediction: {str(e)}")
            return {"error": "Error in prediction."}

    def save_model(self, path):
        """Save the trained model and scaler to a file."""
        model_data = {
            'model': self.model,
            'scaler': self.scaler
        }
        with open(path, 'wb') as file:
            pickle.dump(model_data, file)

    def load_model(self, path):
        """Load the trained model and scaler from a file."""
        with open(path, 'rb') as file:
            model_data = pickle.load(file)
        self.model = model_data['model']
        self.scaler = model_data['scaler']


In [73]:
def extract_features(self, audio_path):
        """Extract features from the audio file."""
        try:
            y, sr = librosa.load(audio_path, duration=5)  # Load audio (5 seconds max)
            
            # Extract features
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
            zero_crossing_rate = librosa.feature.zero_crossing_rate(y)

            # Calculate statistics for each feature
            features = {
                'mfcc_mean': np.mean(mfccs, axis=1),
                'mfcc_std': np.std(mfccs, axis=1),
                'spectral_centroid_mean': np.mean(spectral_centroid),
                'spectral_centroid_std': np.std(spectral_centroid),
                'zero_crossing_rate_mean': np.mean(zero_crossing_rate),
                'zero_crossing_rate_std': np.std(zero_crossing_rate)
            }
            feature_vector = []
            for value in features.values():
                if isinstance(value, np.ndarray):
                    feature_vector.extend(value)
                else:
                    feature_vector.append(value)

            return np.array(feature_vector)
        except Exception as e:
            print(f"Error extracting features from {audio_path}: {str(e)}")
            return None
            

# Using main function

In [71]:
def main():
    # Initialize the StutterDetector
    detector = StutterDetector()

    # Example audio files and labels (Replace these paths with your actual audio files)
    audio_files = [
        r"C:\Users\91750\Desktop\PROJECTS\Stutter Project(Intern Work)\Stutter 1.mp3",  # Replace with actual audio file path
        r"C:\Users\91750\Desktop\PROJECTS\Stutter Project(Intern Work)\Stutter 2.mp3",  # Replace with actual audio file path
        r"C:\Users\91750\Desktop\PROJECTS\Stutter Project(Intern Work)\No Stutter 2.mp3"  # Replace with actual audio file path
    ]
    labels = [1, 1, 0]  # 1 = stutter, 0 = fluent

    # Train the model
    print("Training the model...")
    detector.train(audio_files, labels)

    # Test prediction
    print("\nTesting prediction...")
    audio_path = r"C:\Users\91750\Desktop\PROJECTS\Stutter Project(Intern Work)\No Stutter 1.mp3"# Replace with actual test audio file path
    result = detector.predict(audio_path)

    print("Prediction Results:")
    print(f"Prediction: {result['prediction']}")
    print(f"Confidence: {result['confidence']:.2f}")
    print("Recommendations:")
    for rec in result["recommendations"]:
        print(f"- {rec}")


if __name__ == "__main__":
    main()


Training the model...
Training Accuracy: 1.00
Testing Accuracy: 0.00

Testing prediction...
Prediction Results:
Prediction: fluent
Confidence: 0.59
Recommendations:
- Maintain confidence in your speech.
- Continue practicing speaking regularly to maintain fluency.
- Explore public speaking to further improve fluency and confidence.
